In [3]:
!export CUDA_VISIBLE_DEVICES=0

'export' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.current_device()

0

In [8]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import torch
torch.cuda.is_available()

# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'
# # os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

# torch.cuda.set_device(0)
# print(torch.cuda.current_device())

import yaml
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()
    print(torch.multiprocessing.get_start_method())
#     torch.multiprocessing.set_start_method('spawn',force=True)
    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            # from train_bertclass import Solver
            from Whisper_Biclass_bert_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))

    for idx in range(0,5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}.yaml'#latest7k_4lstm.pth
        # setattr(paras, 'load', f'./ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/bert8k.pth')
        #要用vag01要改load_ckpt
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)


        solver.load_data()
        
#         solver.print_model()
        solver.set_model()
        solver.exec()
        del solver

In [2]:
main()

spawn

Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     


Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTT5-5-16', 'CTT5-2-16', 'CTT5-3-16', 'CTT5-4-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
Mozillacv11Dataset CTT5-2-16 found wav data: 18
Mozillacv11Dataset CTT5-3-16 found wav data: 17
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 69
remove None, then wav data: 69, text len: 69
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0K. 

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-16']
Mozillacv11Dataset CTT5-2-16 found wav data: 18
text len: 18
remove None, then wav data: 18, text len: 18
[VAGDataset] path: data_process, split: ['CTT5-1-16', 'CTT5-5-16', 'CTT5-3-16', 'CTT5-4-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
Mozillacv11Dataset CTT5-5-16 found wav data: 17
Mozillacv11Dataset CTT5-3-16 found wav data: 17
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 68
remove None, then wav data: 68, text len: 68
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0K. 

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-16']
Mozillacv11Dataset CTT5-3-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTT5-1-16', 'CTT5-2-16', 'CTT5-5-16', 'CTT5-4-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
Mozillacv11Dataset CTT5-2-16 found wav data: 18
Mozillacv11Dataset CTT5-5-16 found wav data: 17
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 69
remove None, then wav data: 69, text len: 69
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0K. 

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-16']
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTT5-1-16', 'CTT5-2-16', 'CTT5-3-16', 'CTT5-5-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
Mozillacv11Dataset CTT5-2-16 found wav data: 18
Mozillacv11Dataset CTT5-3-16 found wav data: 17
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 69
remove None, then wav data: 69, text len: 69
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0K. 

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTT5-1-16', 'CTT5-2-16', 'CTT5-3-16', 'CTT5-4-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
Mozillacv11Dataset CTT5-2-16 found wav data: 18
Mozillacv11Dataset CTT5-3-16 found wav data: 17
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 69
remove None, then wav data: 69, text len: 69
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0K. 

In [1]:
# decode about 30 sec
import torch
import os
import pandas as pd

from transformers import BertTokenizer, BertModel
import soundfile as sf

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = BertTokenizer.from_pretrained('shibing624/text2vec-base-chinese')
model = BertModel.from_pretrained('shibing624/text2vec-base-chinese')
sentences = ['如何更换花呗绑定银行卡', '花呗更改绑定银行卡']
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print("Sentence embeddings:")
print(sentence_embeddings)
# path = "data_process/CTTsegment_remove"
# transcriptionlist = pd.DataFrame()
# for f in os.listdir(path):
#     filepath = path + '/' + f
#     if os.path.isfile(filepath) and f.find("wav") != -1:
#         print("filepath: ", filepath)
#         wav, sr = sf.read(filepath)
#         temp = "data_process/CTTsegment_remove/temp.wav"
#         sf.write(temp, wav, 16000)
#         wav, sr = sf.read(temp)
#         print(wav, sr)
#         # import librosa    
#         # wav, sr = librosa.load(filepath, sr=16000) # Downsample 44.1kHz to 8kHz
#         # print(sr)
#         # outofCUDA
#         # input_features = processor(wav, sampling_rate=sr, return_tensors="pt").input_features.to(device)
        
#         input_values = processor(wav, sampling_rate=sr, return_tensors="pt", padding="longest").input_values.to(device)  # Batch size 1
#         print(input_values.shape)
#         # retrieve logits
#         logits = model(input_values).last_hidden_state
        
#         # # take argmax and decode
#         # predicted_ids = torch.argmax(logits, dim=-1)
#         # transcription = processor.batch_decode(predicted_ids)

#         # print(logits)

In [ ]:
# torch.cuda.device_count()
import torch
torch.cuda.is_available()
# torch.cuda.device_count() 

False

In [ ]:
import torch
print(torch.__version__)

2.3.1+cpu


In [ ]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv

In [1]:
import torch
torch.cuda.is_available()

# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'
# # os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

# torch.cuda.set_device(0)
# print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
# import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-bertclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()
    print(torch.multiprocessing.get_start_method())
#     torch.multiprocessing.set_start_method('spawn',force=True)
    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            # from train_bertclass import Solver
            from Whisper_Biclass_bert_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))

    for idx in range(0,5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}.yaml'#latest7k_4lstm.pth
        setattr(paras, 'load', f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/bert-v2-15k-r-final-2.pth')
        #要用vag01要改load_ckpt
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)


        solver.load_data()
#         solver.print_model()
        solver.set_model()
        # solver.exec()
        # solver.validate(idx)
        solver.visualization()
        del solver

In [2]:
main()

spawn

Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTT5-5-16', 'CTT5-2-16', 'CTT5-3-16', 'CTT5-4-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
Mozillacv11Dataset CTT5-2-16 found wav data: 18
Mozillacv11Dataset CTT5-3-16 found wav data: 17
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 69
remove None, then wav data: 69, text len: 69
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=Fals

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


output tensor([0.4526], device='cuda:0', grad_fn=<SelectBackward0>)


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\captum\attr\_utils\visualization.py:338: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\utils\tensorboard\_utils.py:22: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  canvas.draw()


AttributeError: 'BertModel' object has no attribute 'bert'

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import argparse
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from transformers import (
    AutoModelForMaskedLM,
    BertModel,
    BertTokenizer,
    AdamW,
    BertForSequenceClassification
)
from sklearn.metrics import accuracy_score, classification_report

device = 'cuda:0'
model_checkpoint = "google-bert/bert-base-chinese"
bert = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
bert = bert.to(device)
optimizer = AdamW(bert.parameters(), lr=5e-5)

model_version = 'google-bert/bert-base-chinese'
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version)



class Para(object):
    a=1

def list_5fold():
    path_list = []
    for idx in range(5):
        path = "data_process/CTT5-" + str(idx+1) + "-2"
        for f in os.listdir(path):
            file = int(str(f).split("CTT")[0].split('/')[-1])
            path_list.append({"file": file, "5fold": idx+1})
    return path_list

def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print("start")
    
    path_list = pd.DataFrame(list_5fold())
    path_list = path_list.sort_values(by='file', ascending=True)
    # path_list.set_index("file" , inplace=True)
    path_list = path_list.reset_index()
    # print(path_list)

    txt = pd.read_csv('data_process/whisper-v2.csv')
    # txt = txt.replace({'eval': {-1: 1}})
    # print(txt)
    txt["5fold"] = path_list["5fold"]
    # txt["input"] = [str(txt["AD"][i]) + str(txt["transcription"][i]) for i in range(len(txt))]

    feature = txt.drop(columns = ['file', 'AD', '5fold']).astype(str).to_numpy()
    # feature = txt["eval"].to_numpy()
    # feature = np.reshape(feature, (-1, 1))

    # print(feature)

    target = txt["AD"].to_numpy()
    target = np.reshape(target, (-1, 1))
    # print(txt, feature, target)
    
    alldf =pd.DataFrame()
    for fold in range(0,5):
        # 5 fold
        train = []
        valid = []
        t_tra = []
        t_val = []
        name = []
        # y_predicted_val = []
        idx = 0

        for idx in range(feature.shape[0]):
            if(int(txt["5fold"][idx]) != int(str(fold+1))):
                train.append(feature[idx])
                t_tra.append(target[idx])
            else:
                valid.append(feature[idx])
                t_val.append(target[idx])
                # if(feature[idx]>=5):
                #     y_predicted_val.append(0)
                # else:
                #     y_predicted_val.append(1)                    
                name.append(txt["file"][idx])

        train = np.array(train)
        t_tra = np.array(t_tra)
        valid = np.array(valid)
        t_val = np.array(t_val)
        name = np.array(name)
        # name = np.reshape(name, (-1, 1)) 
        
        tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-chinese')
        model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=len(set(torch.tensor(t_tra))))
        print("fold: ", fold)
         
        for i in range(len(train)):
            print(train[i])
            train_encodings = tokenizer(train[i][0], truncation=True, padding=True, max_length=128)
            labels = torch.tensor(t_tra[i]).to(device)

            optimizer.zero_grad()
            input_ids = torch.tensor(train_encodings['input_ids']).to(device)
            attention_mask = torch.tensor(train_encodings['attention_mask']).to(device)
            # labels = item['labels'].to(device)
            print(input_ids, attention_mask, labels)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
        
        model.eval()
        predictions, true_labels = [], []
        for i in range(len(valid)):
            test_encodings = tokenizer(train[i][0], truncation=True, padding=True, max_length=128)
            labels = torch.tensor(t_val[i]).to(device)

            input_ids = torch.tensor(test_encodings['input_ids']).to(device)
            attention_mask = torch.tensor(test_encodings['attention_mask']).to(device)
            # labels = item['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
        
       # 計算準確率
        accuracy = accuracy_score(true_labels, predictions)
        print(f'Accuracy: {accuracy}')

        # 顯示分類報告
        print(classification_report(true_labels, predictions))

#         # ds = DecisionTreeClassifier()
#         # knn = KNeighborsClassifier()
#         # svc = SVC(kernel='poly', C=1, gamma="auto", degree=3, probability=True)
#         svc = SVC(kernel='linear', C=100, gamma="auto", probability=True)
#         # clf = BaggingClassifier(base_estimator=svc, n_estimators=10, random_state=0)
#         clf = make_pipeline(StandardScaler(), svc)

#         # Fit the model to the data
#         clf.fit(train, t_tra.ravel())
#         # print(train, t_tra)
# #         print(reg.coef_)
#         y_predicted = clf.predict(train)
# #         loss = mean_squared_error(t_tra, y_predicted)
#         y_predicted_val = clf.predict(valid)
#         y_predicted_val_pro = clf.predict_proba(valid)
#         y_predicted_val_pro = [row[1] for row in y_predicted_val_pro]

        # feature_import = pd.Series(imp, index=names).nlargest(4).plot(kind='barh')
        # print(feature_import)

        # y_predicted_val = np.reshape(y_predicted_val, (-1, 1)) 
#         loss_val = mean_squared_error(t_val, y_predicted_val)
#         print("loss ", loss, "loss_val", loss_val)

        
        # pipeline = Pipeline([
        #     ('scaler', StandardScaler()), 
        #     ('svc', SVC(probability=True, random_state=1))
        # ])

        # param_grid = {
        #     'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        #     'svc__C': [0.1, 1, 10, 100],
        #     'svc__gamma': ['scale', 'auto'], 
        #     'svc__degree': [2, 3, 4] 
        # }

        # grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)
        # grid_search.fit(train, t_tra.ravel())

        # print("Best Parameters: ", grid_search.best_params_)
        # best_clf = grid_search.best_estimator_
        # y_predicted_val = best_clf.predict(valid)

        # imp = svc.coef_
        # imp = abs(imp[0])
        # print(imp)   
        # names = ['fluency', 'richness', 'correlation', 'grammar', 'complexity']
        # imp,names = zip(*sorted(zip(imp,names)))
        # plt.barh(range(len(names)), imp, align='center')
        # plt.yticks(range(len(names)), names)
        # plt.show()

        # train = np.reshape(train, (-1,))
        # t_tra = np.reshape(t_tra, (-1,))
        # valid = np.reshape(valid, (-1,))
        # t_val = np.reshape(t_val, (-1,))
        # y_predicted_val_pro = np.reshape(y_predicted_val_pro, (-1,))

        # print('train:', train)
        # print(t_tra)
        # print("valid", valid)
        # print(y_predicted_val)

        # plt.scatter(train, t_tra, cmap=plt.cm.coolwarm)
        # plt.xlabel('Sepal length')
        # plt.ylabel('Sepal width')
        # plt.xticks(())
        # plt.yticks(())
        # plt.show()
        
        # print(name, y_predicted_val, t_val)
#         df = pd.DataFrame(
#             {'name': name,
#              'result': y_predicted_val,
#              'truth': t_val,
#              'hyps': y_predicted_val_pro
#             })
#         alldf = pd.concat([alldf, df])#
# #         print(df)
#         # df.to_csv(f'biclass_result/5folds_biclass_{fold+1}_pause_e.csv', index=False)
#     alldf.to_csv("biclass_result/bert-v2-combine.csv", index=False) 
    return True


0


Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\naomi\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable t

In [ ]:
main()

start


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


fold:  0
['我看到一個小男生站在椅子上想要拿餅乾盒然後他看起來快要跌倒然後有一個小女孩在下面想要接他從餅乾盒拿出來的東西櫃子是打開的然後有一位女性正在洗碗朝前面洗碗然後水龍頭是打開的水已經滿出來了 流到地上然後有窗戶窗簾半開窗戶是打開的外面可以看到房子外有一些草木然後琉璃台上面還有幾個碗盤看不出來洗過了沒有這應該是一個床']
tensor([ 101, 2769, 4692, 1168,  671,  943, 2207, 4511, 4495, 4991, 1762, 3488,
        2094,  677, 2682, 6206, 2897, 7619,  746, 4665, 4197, 2527,  800, 4692,
        6629,  889, 2571, 6206, 6649,  948, 4197, 2527, 3300,  671,  943, 2207,
        1957, 2111, 1762,  678, 7481, 2682, 6206, 2970,  800, 2537, 7619,  746,
        4665, 2897, 1139,  889, 4638, 3346, 6205, 3602, 2094, 3221, 2802, 7274,
        4638, 4197, 2527, 3300,  671,  855, 1957, 2595, 3633, 1762, 3819, 4813,
        3308, 1184, 7481, 3819, 4813, 4197, 2527, 3717, 7983, 7531, 3221, 2802,
        7274, 4638, 3717, 2347, 5195, 4021, 1139,  889,  749, 3837, 1168, 1765,
         677, 4197, 2527, 3300, 4970, 2786, 4970, 5088, 1288, 7274, 4970, 2786,
        3221, 2802, 7274, 4638, 1912, 7481, 1377,  809, 4692, 1168, 2791, 2094,
        1912, 3300,  671,  763,

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
from sklearn import metrics

llama_v2_hyps = llama_v2['hyps'].tolist()
llama_v2_result = llama_v2['result'].tolist()
llama_v2_truth = llama_v2['truth'].tolist()

llama_v2_wrong = llama_v2.iloc[np.where(llama_v2['result'] != llama_v2['truth'])]
print(llama_v2_wrong)

target_names = ['non-patient', 'patient']
clf_report = metrics.classification_report(llama_v2_truth, llama_v2_result, target_names=target_names)
print(clf_report)

tn, fp, fn, tp = metrics.confusion_matrix(llama_v2_truth, llama_v2_result).ravel()
print('sensitivuty: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))

fpr, tpr, thresholds = metrics.roc_curve(np.array(llama_v2_truth), np.array(llama_v2_hyps), pos_label=1)
print('AUC: ',metrics.auc(fpr, tpr))
fpr1, tpr1, thresholds1 = metrics.roc_curve(np.array(llama_v2_truth[0:19]), np.array(llama_v2_hyps[0:19]), pos_label=1)
auc1 = metrics.auc(fpr1, tpr1)
fpr2, tpr2, thresholds2 = metrics.roc_curve(np.array(llama_v2_truth[19:37]), np.array(llama_v2_hyps[19:37]), pos_label=1)
auc2 = metrics.auc(fpr2, tpr2)
fpr3, tpr3, thresholds3 = metrics.roc_curve(np.array(llama_v2_truth[37:54]), np.array(llama_v2_hyps[37:54]), pos_label=1)
auc3 = metrics.auc(fpr3, tpr3)
fpr4, tpr4, thresholds4 = metrics.roc_curve(np.array(llama_v2_truth[54:71]), np.array(llama_v2_hyps[54:71]), pos_label=1)
auc4 = metrics.auc(fpr4, tpr4)
fpr5, tpr5, thresholds5 = metrics.roc_curve(np.array(llama_v2_truth[71:88]), np.array(llama_v2_hyps[71:88]), pos_label=1)
auc5 = metrics.auc(fpr5, tpr5)
print('AUC: ', auc1, auc2, auc3, auc4, auc5)
print("AUC_avg: ", (auc1+auc2+auc3+auc4+auc5)/5)

y_test = [bool(int(x)) for x in llama_v2_truth]
y_pred = [bool(int(x)) for x in llama_v2_result]
metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap="Blues")